# 百順
- http://www.ptrtour.com.tw/web_3/major_tour_list.asp?pageid=1

In [29]:
#測試soup.select
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors

sample='\
<table>\
    <tr tour1>\
        <td 1="出發日期"></td>\
        <td ></td>\
        <td 3="團名">\
            <a href="tour1url">title1</a>\
            <span class=list_memo>結團</span> </td>\
        <td 4="word">\
            <a href="word1url">word1</a> </td>\
        ... </tr>\
    <tr tour2>\
        <td 1="出發日期"></td>\
        <td ></td>\
        <td 3="團名">\
            <a href="tour2url">title2</a>\
            <span class=list_memo>結團</span> </td>\
        <td 4="word">\
            <a href="word2url...">word2</a> </td>\
        ... </tr>\
    <tr tour3>\
        <td 1="出發日期"></td>\
        <td  ></td>\
        <td 3="團名">\
            <a href="tour3url">title2</a>\
            <span class=list_memo>結團</span> </td>\
        <td 4="word">\
            <a href="word3url...">"&nbsp;"<img src="images/page_next.gif"> </a> </td>\
        ...</tr>\
<div id=list_chang_page>\
    <a href="上一頁">"&nbsp;"<img src="images/page_previous.gif"> </a>\
    <a></a>\
    <a></a>\
    ...\
    <a href="下一頁">"&nbsp;"<img src="images/page_next.gif"> </a>\
</div>'
    
#指定第幾個孩子: tr > td:nth-of-type(3) 指定tr的孩子td第三個
from bs4 import BeautifulSoup 
soup = BeautifulSoup(sample,'html.parser')
titles=[]
hrefs=[]
for ht in soup.select('table tr td[2]'):    #td[2]不能,需用td:nth-of-type(3)
    print ht #無值
for ht in soup.select('table tr td:nth-of-type(3)'):    #w/o '>'：只有第一個   
    print ht
for ht in soup.select('table tr > td:nth-of-type(3)'):    #w/ '>'：符合都有 
    print ht    
print "---------------------------------------------------------------------"
 
    
#某屬性的屬性質含有特定字串    
for h in soup.select('table tr > td:nth-of-type(4) a'):
    if 'word3url' in h['href']:
        print h
print "---------------------------------------------------------------------"


#測試空白在tag.text
for img in soup.select('#list_chang_page a img'):
    if 'images/page_next.gif' in img["src"]:
        print img
#改寫成while(小心測試無窮迴圈)
#while len([img for img in soup.select('#list_chang_page a img') if 'images/page_next.gif' in img["src"] ]) > 0:
#    print img
    
a=[1,2,3,4]
print len(a)


<td 3="團名"> <a href="tour1url">title1</a> <span class="list_memo">結團</span> </td>
<td 3="團名"> <a href="tour1url">title1</a> <span class="list_memo">結團</span> </td>
<td 3="團名"> <a href="tour2url">title2</a> <span class="list_memo">結團</span> </td>
<td 3="團名"> <a href="tour3url">title2</a> <span class="list_memo">結團</span> </td>
---------------------------------------------------------------------
<a href="word3url...">" "<img src="images/page_next.gif"> </img></a>
---------------------------------------------------------------------
<img src="images/page_next.gif"> </img>
4


In [97]:
#測試.append()
a=["aaa","bbb","ccc","ddd"]
A=[]
for i in xrange(0,3):
    A.append(a[i])
print A[0]

aaa


### 行程列表
- 載入後條件篩選:滿團, 標題相同
- 輸出hrefs到檔案

In [101]:
#一頁
'''
<table>    
    <tbody>#tbody不適用
        <tr tour1>
            <td0 出發日期>
            <td1 >
            <td2 團名>
                <a href="...url...">title</a>
                <span class=list_memo>結團</span>
            ...
        <tr tour2>
        <tr tour3>
'''

#載入第一頁
from bs4 import BeautifulSoup 
import requests
res  = requests.get('http://www.ptrtour.com.tw/web_3/major_tour_list.asp?pageid=1')
res.encoding='big5'
soup = BeautifulSoup(res.text)
#print res.text
titles=[]
hrefs=[]

#篩選：額滿
for ele in soup.select('.list_table tr')[1:-1]:
    #print ele
    #print "---------------------------------------------------------------------"
    #for available in ele.select('td:nth-of-type(12) > a'):
    #    print  available
    if len([ available for available in ele.select('td:nth-of-type(12) > a') ])>0:
        #print ele        
        for t in ele.select('td:nth-of-type(3) > a'):
            #print t.text 
            #print "http://www.ptrtour.com.tw/web_3/"+t["href"] 
            titles.append(t.text)
            hrefs.append( "http://www.ptrtour.com.tw/web_3/"+t["href"] )
print len(hrefs)
print titles[4]
print hrefs[4]


#篩選：重複行程排序從資料庫指令操作



#檢查有無下一頁
for img in soup.select('#list_chang_page a > img'):
    if 'images/page_next.gif' in img["src"]:
        print img
#改寫成while
#while len([ img for img in soup.select('#list_chang_page a > img') if 'images/page_next.gif' in img["src"] ]) > 0:
#    print len(img) 

10
日本特殊團
http://www.ptrtour.com.tw/web_3/content_show_sql_detail.asp?travel_no=NGO02CX160605T
<img align="absmiddle" border="0" src="images/page_next.gif"/>


In [117]:
#測試：改寫selenium動態下一頁：//div[@id='list_chang_page']/table/tbody/tr/td/a[17]/img改為/a/img[@src='images/page_next.gif']
#find_element_by_xpath:http://selenium-python.readthedocs.io/locating-elements.html

# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
from bs4 import BeautifulSoup 
import json

browser = webdriver.Firefox()
browser.get("http://www.ptrtour.com.tw/")
Select(browser.find_element_by_name("nation_code")).select_by_visible_text(u"日本")
browser.find_element_by_name("submit").click()
soup = BeautifulSoup(browser.page_source)
while len([ img for img in soup.select('#list_chang_page a > img') if 'images/page_next.gif' in img["src"] ]) > 0:
    browser.find_element_by_xpath("//div[@id='list_chang_page']/table/tbody/tr/td/a/img[@src='images/page_next.gif']").click()
    soup = BeautifulSoup(browser.page_source)
print len(titles)

1


In [29]:
#多頁
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
from bs4 import BeautifulSoup 
import json
import copy

browser = webdriver.Firefox()
browser.get("http://www.ptrtour.com.tw/")
Select(browser.find_element_by_name("nation_code")).select_by_visible_text(u"日本")
browser.find_element_by_name("submit").click()
soup = BeautifulSoup(browser.page_source,'html.parser')
titles=[]
hrefs=[]
prodNos=[]

#改寫selenium
while len([ img for img in soup.select('#list_chang_page a > img') if 'images/page_next.gif' in img["src"] ]) > 0:
    for ele in soup.select('.list_table tr')[1:-1]:
        if len([ available for available in ele.select('td:nth-of-type(12) > a') ])>0:      
            for t in ele.select('td:nth-of-type(3) > a'):
                titles.append(t.text)
                hrefs.append( "http://www.ptrtour.com.tw/web_3/"+t["href"] )
                prodNos.append(t["href"].split('travel_no=')[-1])             
                
    try:
        browser.find_element_by_xpath("//div[@id='list_chang_page']/table/tbody/tr/td/a/img[@src='images/page_next.gif']").click()
        soup = BeautifulSoup(browser.page_source)
    except NoSuchElementException as msg:
        print "end"
        break  

#print len(hrefs)
#print titles[4]
print hrefs[4]
print prodNos[4]

#寫出到.json
with open('E:/ETL/practice/prttravel_hrefs.json' , 'w') as f:
    json.dump(hrefs, f)

http://www.ptrtour.com.tw/web_3/content_show_sql_detail.asp?travel_no=SPK05GE160607A
SPK05GE160607A


IOError: [Errno 22] invalid mode ('w') or filename: 'E:/ETL/practice/prttravel\xe7\x99\xbe\xe9\xa0\x86_hrefs.json'

### 日程內容

In [58]:
#re.sub(pattern, repl, string, count=0, flags=0):留住中文
#http://www.crifan.com/python_re_sub_detailed_introduction/comment-page-1/
from bs4 import BeautifulSoup 
import requests
def removePunctuation(source):
    pattern=u'([^\u4e00-\u9fff]+)'    #^:在[候選集]內表示非中文
    repl=re.sub(pattern,' ',source)    #非中文用'空白'取代
    return repl

In [60]:
#一筆
from bs4 import BeautifulSoup 
import requests
res  = requests.get('http://www.ptrtour.com.tw/web_3/content_show_sql_detail.asp?travel_no=NGO05CX160609A')
res.encoding='big5'
soup = BeautifulSoup(res.text)
import re
import json
#print repr(res.text)

route={}
days = soup.select('.day_no')
locats = soup.select('.day_title')
print repr(days[4].text)
print locats[4].text
print repr(locats[4].text)

for j in xrange(0,len(days)):
    d = re.search('\d+',days[j].text)
    day = "day"+d.group(0)
    #route[day]=locats[j].text.split('\r\n')[1].strip() #.strip():切兩端
    route[day]=removePunctuation(locats[j].text).strip()
    
print route.keys()[4]
print route.values()[4]

u'\u7b2c\xa05\xa0\u5929'

    名古屋－【名古屋∼市區自由活動】∼日本最初集約電波電視塔、市民廣場久屋大通公園、流行發源地榮町－【中部國際機場】∼風靡全球人氣品牌衣飾、遠近馳名土產雜貨、精緻和風藝品／桃園國際機場－
    
    NGO/TPE　CX531　16:50~19:00    
u'\r\n    \u540d\u53e4\u5c4b\uff0d\u3010\u540d\u53e4\u5c4b\u223c\u5e02\u5340\u81ea\u7531\u6d3b\u52d5\u3011\u223c\u65e5\u672c\u6700\u521d\u96c6\u7d04\u96fb\u6ce2\u96fb\u8996\u5854\u3001\u5e02\u6c11\u5ee3\u5834\u4e45\u5c4b\u5927\u901a\u516c\u5712\u3001\u6d41\u884c\u767c\u6e90\u5730\u69ae\u753a\uff0d\u3010\u4e2d\u90e8\u570b\u969b\u6a5f\u5834\u3011\u223c\u98a8\u9761\u5168\u7403\u4eba\u6c23\u54c1\u724c\u8863\u98fe\u3001\u9060\u8fd1\u99b3\u540d\u571f\u7522\u96dc\u8ca8\u3001\u7cbe\u7dfb\u548c\u98a8\u85dd\u54c1\uff0f\u6843\u5712\u570b\u969b\u6a5f\u5834\uff0d\r\n    \r\n    NGO/TPE\u3000CX531\u300016:50~19:00    '
day1
桃園國際機場 名古屋中部國際機場 伊勢灣海上人工島 保護環境之設計 行經東海北陸自行車道 立山黑部之旅就此展開 勝山


In [62]:
#一筆:測試日程抓不出來的行程
from bs4 import BeautifulSoup 
import requests
res  = requests.get('http://www.ptrtour.com.tw/web_3/content_show_sql_detail.asp?travel_no=FUK06CX160811K')
res.encoding='big5'
soup = BeautifulSoup(res.text)
import re
import json
#print repr(res.text)

route={}
days = soup.select('.day_no')
locats = soup.select('.day_title')
print days[4].text
print locats[4].text

for j in xrange(0,len(days)):
    d = re.search('\d+',days[j].text)
    day = "day"+d.group(0)
    #route[day]=locats[j].text.split('\r\n')[1].strip() #.strip():切兩端
    route[day]=removePunctuation(locats[j].text).strip()
    
print route.keys()[4]
print route.values()[4]

第 5 天

    湯本溫泉－【大河劇『花燃』拍攝地∼萩城下町、松下村塾】∼世界文化遺產、日本維新教育改革園地－【秋吉台】∼特別指定天然喀斯特地形紀念物－【免稅店】－【博多】∼街町自主逛遊、天神精品商城、街邊推車部隊、悠閒親富孝通、中洲夜屋台、浪漫露天餐飲繁華街、九州美食蒐羅網－    
day3
豪斯登堡 光明禪寺 九州唯一 枯山水 石庭 太宰府天滿宮 江戶古街道 心字池 太鼓橋 櫻門 筆塚 學問之神菅原道真庇護 花崗岩製中世大鳥居 增長智慧財運御神牛 小倉城 日本唯一唐造天守閣 白野江植物公園 四季之丘 海景山頂廣場 北九州


In [64]:
#匯入.json
import json
with open('E:/ETL/practice/prttravel_hrefs.json' , 'r') as f:
    fhrefs = json.load(f)
print len(fhrefs)

216


In [66]:
#多筆
from bs4 import BeautifulSoup 
import requests
import re
import json

routes=[]
for h in xrange(0,len(fhrefs)):
    res = requests.get(fhrefs[h])
    res.encoding='big5'
    soup = BeautifulSoup(res.text)
    #print soup

    route={}
    days = soup.select('.day_no')
    locats = soup.select('.day_title')
   
    try:
        for j in xrange(0,len(days)):
            d = re.search('\d+',days[j].text)
            day = "day"+d.group(0)
            route[day]=locats[j].text.split('\r\n')[1].strip()
        routes.append(route)
    except IndexError as msg: #取出數量不同的hrefs
        print fhrefs[h]
        continue         
        
print len(routes)

#寫出到.json
with open('E:/ETL/practice/prttravel_routes.json' , 'w') as f:
    json.dump(routes, f)

ConnectionError: HTTPConnectionPool(host='www.ptrtour.com.tw', port=80): Max retries exceeded with url: /web_3/content_show_sql_detail.asp?travel_no=SPK05GE160628A (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0000000008773898>: Failed to establish a new connection: [Errno 10060] \xb3s\xbdu\xb9\xc1\xb8\xd5\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xb9\xef\xb6H\xa6\xb3\xa4@\xacq\xae\xc9\xb6\xa1\xa8\xc3\xa5\xbc\xa5\xbf\xbdT\xa6^\xc0\xb3\xa1A\xa9\xce\xacO\xb3s\xbdu\xab\xd8\xa5\xdf\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xaa\xba\xa5D\xbe\xf7\xb5L\xaak\xa6^\xc0\xb3\xa1C',))

In [18]:
#測試:迭代印出字典,搭配迴圈
for rr in xrange(0,2):
    route=routes[rr]
    for r in route:
        print r, route[r]
    print "-------------------------------------"

day4 札幌－【北海道廳舊本廳舍－時計台鐘樓－大通公園】－【北海道神宮】∼開拓判官祈福御守、六花亭獨售判官餅－【免稅店】∼精挑細選拌手禮品、姊妹婆媽大小包－【小樽∼電影情書拍攝地】∼羅曼蒂克運河、紅磚倉庫群、世界工藝北一硝子館、匠心獨具音樂盒博物館、歐風蒸氣鐘、秘傳洋菓人氣店家－【定山溪溫泉∼人氣溫泉旅館250選──五星之宿】 ∼美泉浴湯、鹽分水質美人湯、童話河童之故鄉、豪華豐盛筵席－
day5 定山溪溫泉－【白色戀人公園】∼夢幻歐風城堡、浪漫英式庭園、童趣人偶時計塔、甜點物語－【千歲空港】∼北國最大機場、哆啦Ａ夢雲上樂園∼銅鑼燒溜滑梯，繽紛遊樂區、ROYCE巧克力工房∼巧克力櫥窗派對、Steiff玩偶動物園∼泰迪熊、Hello Kitty千歲空港獨家商品／桃園國際機場－
day2 旭川－【熊出沒注意！特贈價值¥600熊牧場入場】∼憨傻棕熊逗趣表演、翻滾招手百變姿態－【石狩川】∼北海道的母親河、北海道遺產－【大雪湖、石北峠】－【網走刑務所】∼雄偉堅固紅磚門、伴賞鄂霍次克海－【網走國立公園、濤沸湖溼地∼道東白鳥飛來地】∼沙丘潟湖、多種鳥類棲息－【大空芝櫻公園∼第33回芝櫻05/03－06/05限定∼櫻色粉紫地毯】－【阿寒湖溫泉∼獨家親密觸碰綠球藻精靈合影、特贈：迎賓銘菓、溫馨迎賓咖啡】∼禪境露天風呂、自然芬多精瀰漫－
day3 阿寒湖溫泉－【特贈：阿寒快艇遊湖∼世界遺產阿寒湖、米其林三星景點】∼春語日和、幸福物語－【丹頂鶴公園∼特贈：價值¥1000 GO！GO！PASS】∼精靈舞者優美姿態－【釧路溼原展望台】－【21世紀足寄驛舍】∼駐足銀河鐵道舊時路、思緒列車夢迴月台側－【日高樹海、山脈、十勝平原】－【札幌】∼夜遊狸小路、薄野鬧區、拉麵橫町、藥�小舖－
day1 桃園國際機場∼【特贈：百順貴賓專屬餐點、樂樂龍提袋】／旭川空港－【旭川拉麵村∼八大拉麵名家、結緣拉麵神社】∼蒐羅北海道在地美味、大神庇祐情綿意長久、三大拉麵筷匙飛舞－【男山酒造】∼清酒試飲、繪卷觀覽、名酒選購－【旭川】∼四季風貌景色醉人、空中大浴場展望星空－
-------------------------------------
day2 岐阜－【白川鄉合掌村】∼世界文化遺產、童話山村田園小屋－【高山飛驒上三町古街】∼工藝寶藏庫－【信州仙境上高地】∼自然瑰寶保護區、河童橋展望穗高連峰、明神嶽、梓川、日本

In [63]:
#?!!!數量不同
print len(fhrefs)
print len(titles)
print len(routes) #回頭測試日程內容

216
216
198


In [27]:
with open('E:/ETL/practice/prttravel百順_routes.json' , 'r') as f:
    prttravel_Routes = json.load(f)
    print type(prttravel_Routes)
    
#檢視.json
print json.dumps(prttravel_Routes, encoding='UTF-8', ensure_ascii=False, sort_keys=True)

<type 'list'>
[{"day1": "桃園國際機場∼【特贈：百順貴賓專屬餐點、樂樂龍提袋】／旭川空港－【旭川拉麵村∼八大拉麵名家、結緣拉麵神社】∼蒐羅北海道在地美味、大神庇祐情綿意長久、三大拉麵筷匙飛舞－【男山酒造】∼清酒試飲、繪卷觀覽、名酒選購－【旭川】∼四季風貌景色醉人、空中大浴場展望星空－", "day2": "旭川－【熊出沒注意！特贈價值¥600熊牧場入場】∼憨傻棕熊逗趣表演、翻滾招手百變姿態－【石狩川】∼北海道的母親河、北海道遺產－【大雪湖、石北峠】－【網走刑務所】∼雄偉堅固紅磚門、伴賞鄂霍次克海－【網走國立公園、濤沸湖溼地∼道東白鳥飛來地】∼沙丘潟湖、多種鳥類棲息－【大空芝櫻公園∼第33回芝櫻05/03－06/05限定∼櫻色粉紫地毯】－【阿寒湖溫泉∼獨家親密觸碰綠球藻精靈合影、特贈：迎賓銘菓、溫馨迎賓咖啡】∼禪境露天風呂、自然芬多精瀰漫－", "day3": "阿寒湖溫泉－【特贈：阿寒快艇遊湖∼世界遺產阿寒湖、米其林三星景點】∼春語日和、幸福物語－【丹頂鶴公園∼特贈：價值¥1000 GO！GO！PASS】∼精靈舞者優美姿態－【釧路溼原展望台】－【21世紀足寄驛舍】∼駐足銀河鐵道舊時路、思緒列車夢迴月台側－【日高樹海、山脈、十勝平原】－【札幌】∼夜遊狸小路、薄野鬧區、拉麵橫町、藥�小舖－", "day4": "札幌－【北海道廳舊本廳舍－時計台鐘樓－大通公園】－【北海道神宮】∼開拓判官祈福御守、六花亭獨售判官餅－【免稅店】∼精挑細選拌手禮品、姊妹婆媽大小包－【小樽∼電影情書拍攝地】∼羅曼蒂克運河、紅磚倉庫群、世界工藝北一硝子館、匠心獨具音樂盒博物館、歐風蒸氣鐘、秘傳洋菓人氣店家－【定山溪溫泉∼人氣溫泉旅館250選──五星之宿】 ∼美泉浴湯、鹽分水質美人湯、童話河童之故鄉、豪華豐盛筵席－", "day5": "定山溪溫泉－【白色戀人公園】∼夢幻歐風城堡、浪漫英式庭園、童趣人偶時計塔、甜點物語－【千歲空港】∼北國最大機場、哆啦Ａ夢雲上樂園∼銅鑼燒溜滑梯，繽紛遊樂區、ROYCE巧克力工房∼巧克力櫥窗派對、Steiff玩偶動物園∼泰迪熊、Hello Kitty千歲空港獨家商品／桃園國際機場－"}, {"day1": "台北－名古屋－田縣神社一名古屋－", "day2": "岐阜－【白川鄉合掌村】∼世界文化遺產、童話山村田園小屋－【高山飛驒上三町古街】∼工

# 完整版

In [63]:
#行程列表

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
from bs4 import BeautifulSoup 
import json
import copy

browser = webdriver.Firefox()
browser.get("http://www.ptrtour.com.tw/")
Select(browser.find_element_by_name("nation_code")).select_by_visible_text(u"日本")
browser.find_element_by_name("submit").click()
soup = BeautifulSoup(browser.page_source,'html.parser')
titles=[]
hrefs=[]
prodNos=[]

#改寫selenium
while len([ img for img in soup.select('#list_chang_page a > img') if 'images/page_next.gif' in img["src"] ]) > 0:
    for ele in soup.select('.list_table tr')[1:-1]:
        if len([ available for available in ele.select('td:nth-of-type(12) > a') ])>0:      
            for t in ele.select('td:nth-of-type(3) > a'):
                titles.append(t.text)
                hrefs.append( "http://www.ptrtour.com.tw/web_3/"+t["href"] )
                prodNos.append(t["href"].split('travel_no=')[-1])       
    try:
        browser.find_element_by_xpath("//div[@id='list_chang_page']/table/tbody/tr/td/a/img[@src='images/page_next.gif']").click()
        soup = BeautifulSoup(browser.page_source)
    except NoSuchElementException as msg:
        print "end"
        break  

#寫出到.json
with open('E:/ETL/practice/prttravel_hrefs.json' , 'w') as f:
    json.dump(hrefs, f)

#匯入.json
import json
with open('E:/ETL/practice/prttravel_hrefs.json' , 'r') as f:
    fhrefs = json.load(f)
print len(fhrefs)

218


In [65]:
#日程內容

routes=[]
for h in xrange(0,len(fhrefs)):
    res = requests.get(fhrefs[h])
    res.encoding='big5'
    soup = BeautifulSoup(res.text)
    #print soup

    route={}
    days = soup.select('.day_no')
    locats = soup.select('.day_title')
    
    try:
        for j in xrange(0,len(days)):
            d = re.search('\d+',days[j].text)
            day = "day"+d.group(0)
            route[day]=removePunctuation(locats[j].text).strip()
        routes.append(route)
    except IndexError as msg: #取出數量不同的hrefs
        print fhrefs[h]
        continue  
        
#寫出到.json
with open('E:/ETL/practice/prttravel_routes.json' , 'w') as f:
    json.dump(routes, f)

In [66]:
#檢查筆數是否一樣

print len(fhrefs)
print len(titles)
print len(routes)

218
218
218


In [38]:
with open('E:/ETL/practice/prttravel_routes.json' , 'r') as f:
    prttravel_Routes = json.load(f)
    print type(prttravel_Routes)
    
#檢視.json
print json.dumps(prttravel_Routes, encoding='UTF-8', ensure_ascii=False, sort_keys=True)

<type 'list'>
[{"day1": "台北－名古屋－田縣神社一名古屋－", "day2": "岐阜－【白川鄉合掌村】∼世界文化遺產、童話山村田園小屋－【高山飛驒上三町古街】∼工藝寶藏庫－【信州仙境上高地】∼自然瑰寶保護區、河童橋展望穗高連峰、明神嶽、梓川、日本阿爾卑斯山環抱仙履之地－【野尻湖】∼長毛象化石發現地－斑尾高原∼【40平方米豪華套房、星光露天風呂、室內溫水泳池】∼【北國長腳蟹、無限生魚片握壽司、信州有機蔬果、季節限定特製料理、獨家晚餐飲品】－"}, {"day1": "台北－名古屋－田縣神社一名古屋－", "day2": "岐阜－【白川鄉合掌村】∼世界文化遺產、童話山村田園小屋－【高山飛驒上三町古街】∼工藝寶藏庫－【信州仙境上高地】∼自然瑰寶保護區、河童橋展望穗高連峰、明神嶽、梓川、日本阿爾卑斯山環抱仙履之地－【野尻湖】∼長毛象化石發現地－斑尾高原∼【40平方米豪華套房、星光露天風呂、室內溫水泳池】∼【北國長腳蟹、無限生魚片握壽司、信州有機蔬果、季節限定特製料理、獨家晚餐飲品】－"}, {"day1": "桃園國際機場∼【特贈：百順貴賓專屬餐點、樂樂龍提袋】／函館空港－【聖女修道院】∼靜謐歐風庭院、日本最古修道院－【金森倉庫群∼赤煉瓦港灣、古樸歷史廣場、明治郵便局、幸福之鐘、西波止場】∼藍色港灣異國風情、紅磚砌屋儲藏歐風－【函館璀璨夜景∼米其林評選三星景點】∼點點惑星夜光寶盒、腳下群星璀璨閃耀－【湯之川溫泉】∼津輕海峽點點漁火、礦物湯泉療癒身心－", "day2": "湯之川溫泉－【函館朝市】∼海洋食堂市民廚房、高檔漁貨主廚最愛－【昆布館】∼海中蔬菜之王、甘甜鹹變化多樣、鮮味美高湯愛將－【大沼國定公園】∼新日本三景、橋繫百島、野鳥天堂、火山島群綿延、風依水面湖灣闊－【SAIRO洞爺支笏展望台】∼第九大不凍洞爺湖、連峰有珠活火山－【望羊中山】∼遠眺北國富士羊蹄山－【札幌】∼夜遊狸小路、薄野鬧區、拉麵橫町、藥�小舖－", "day3": "札幌－【北海道廳舊本廳舍－時計台鐘樓－大通公園】－【北海道神宮】∼開拓判官祈福御守、六花亭獨售判官餅－【免稅店】∼精挑細選拌手禮品、姊妹婆媽大小包－【日高樹海、山脈－十勝平原】－【21世紀足寄驛舍】∼駐足銀河鐵道舊時路、思緒列車夢迴月台側－【阿寒湖溫泉∼綠球藻精靈親密接觸合影留念、特贈：迎賓銘菓､溫馨迎賓咖啡】∼

In [70]:
#放入字典

tour={}
pktour={}
pktours=[]
for i in xrange(0, len(fhrefs)):
    pktour['agency']='prttravel百順'
    pktour['prodNo']=prodNos[i]
    pktour['href']=fhrefs[i]
    pktour['title']=titles[i]
    pktour['tour']=routes[i]
    
    pktourTemp = pktour.copy()    #.copy(): pktour[]為參照位置,值不同丟入同一參照位值則值會更新只留最新一筆,故需要複製
    pktours.append(pktourTemp)    #使用.append放入list, 否則會出現IndexError: list assignment index out of range
        
#輸出為json檔
startravel_jpPkTours={} 
startravel_jpPkTours['prttravel']=pktours
with open('E:/ETL/practice/prttravel.json' , 'w') as f:
    json.dump(startravel_jpPkTours, f)